In [2]:
# in google colab uncomment this

import os

os.system('apt-get install -y xvfb')
os.system('wget https://raw.githubusercontent.com/yandexdataschool/Practical_DL/fall18/xvfb -O ../xvfb')
os.system('apt-get install -y python-opengl ffmpeg')
os.system('pip install pyglet==1.2.4')

os.system('python -m pip install -U pygame --user')

print('setup complete')

# XVFB will be launched if you run on a server
import os
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY")) == 0:
    !bash ../xvfb start
    %env DISPLAY = : 1

setup complete
Starting virtual X frame buffer: Xvfb.
env: DISPLAY=: 1


In [4]:
import sys
!git clone https://github.com/denklewer/yandex-rl-tasks.git
sys.path.append(os.path.join("", 'yandex-rl-tasks')) # To find local version of the library


fatal: destination path 'yandex-rl-tasks' already exists and is not an empty directory.


# Implementing Advantage-Actor Critic (A2C)

In this notebook you will implement Advantage Actor Critic algorithm that trains on a batch of Atari 2600 environments running in parallel. 

Firstly, we will use environment wrappers implemented in file `atari_wrappers.py`. These wrappers preprocess observations (resize, grayscal, take max between frames, skip frames and stack them together) and rewards. Some of the wrappers help to reset the environment and pass `done` flag equal to `True` when agent dies.
File `env_batch.py` includes implementation of `ParallelEnvBatch` class that allows to run multiple environments in parallel. To create an environment we can use `nature_dqn_env` function. Note that if you are using 
PyTorch and not using `tensorboardX` you will need to implement a wrapper that will log **raw** total rewards that the *unwrapped* environment returns and redefine the implemention of `nature_dqn_env` function here. 



In [0]:
import numpy as np
from util.atari_wrappers_a2c import nature_dqn_env



env = nature_dqn_env("SpaceInvadersNoFrameskip-v4", nenvs=8)
obs = env.reset()
assert obs.shape == (8, 84, 84, 4)
assert obs.dtype == np.uint8
n_actions = env.action_space.n

Next, we will need to implement a model that predicts logits and values. It is suggested that you use the same model as in [Nature DQN paper](https://web.stanford.edu/class/psych209/Readings/MnihEtAlHassibis15NatureControlDeepRL.pdf) with a modification that instead of having a single output layer, it will have two output layers taking as input the output of the last hidden layer. **Note** that this model is different from the model you used in homework where you implemented DQN. You can use your favorite deep learning framework here. We suggest that you use orthogonal initialization with parameter $\sqrt{2}$ for kernels and initialize biases with zeros. 

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# those who have a GPU but feel unfair to use it can uncomment:
# device = torch.device('cpu')

In [0]:
def conv2d_size_out(size, kernel_size=3, stride=2):
    """
    common use case:
    cur_layer_img_w = conv2d_size_out(cur_layer_img_w, kernel_size, stride)
    cur_layer_img_h = conv2d_size_out(cur_layer_img_h, kernel_size, stride)
    to understand the shape for dense layer's input
    """
    return (size - (kernel_size - 1) - 1) // stride + 1


class Flatten(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, x):
        return x.view(x.size(0), -1)

In [0]:
import math
def weights_init(m):
    if type(m) == nn.Linear or type(m) == nn.Conv2d:
        nn.init.orthogonal(m.weight.data, math.sqrt(2.0))
        if m.bias is not None:
            m.bias.data.fill_(0)

class Agent(nn.Module):
    def __init__(self, name, state_shape, n_actions, reuse=False):
        """A simple actor-critic agent"""
        super(Agent, self).__init__()

        # Prepare neural network architecture
        # Your code here: prepare any necessary layers, variables, etc.
        self.n_actions = n_actions
        self.state_shape = state_shape
        print(state_shape)
        n_env, h, w, c = state_shape

        self.common = nn.Sequential(
            nn.Conv2d(state_shape[0], 32, kernel_size=3, stride=2),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, stride=2),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, stride=2),
            nn.ReLU(),
            Flatten()
        )

        convw = conv2d_size_out(conv2d_size_out(conv2d_size_out(w)))
        convh = conv2d_size_out(conv2d_size_out(conv2d_size_out(h)))
        linear_input_size = convw * convh * 32
        # we want to flatten given images to verctor for dense layers.
        print("linear input size = ", linear_input_size)
        self.fc = nn.Sequential(
            nn.Linear(linear_input_size, 128),
            nn.ReLU()
        )

        self.Vs = nn.Linear(128, 1)
        self.logits = nn.Linear(128, n_actions)

        self.common.apply(weights_init)
        self.fc.apply(weights_init)
        self.Vs.apply(weights_init)
        self.logits.apply(weights_init)

    def forward(self, state_t):
        """
        Takes agent's previous step and observation, 
        returns next state and whatever it needs to learn (tf tensors)
        """
        # Apply neural network
        ### Your code here: apply agent's neural network to get policy logits and state values.
        state_t = self.common(state_t)
        state_t = self.fc(state_t)
        logits = self.logits(state_t)
        state_value = self.Vs(state_t)[:, 0]
        return logits, state_value


You will also need to define and use a policy that wraps the model. While the model computes logits for all actions, the policy will sample actions and also compute their log probabilities.  `policy.act` should return a dictionary of all the arrays that are needed to interact with an environment and train the model.
 Note that actions must be an `np.ndarray` while the other
tensors need to have the type determined by your deep learning framework. 

In [0]:
class Policy:
    def __init__(self, model):
        self.model = model
    def sample_actions(self, logits):
        """pick actions given numeric agent outputs (np arrays)"""
        
        policy = np.exp(logits) / np.sum(np.exp(logits), axis=-1, keepdims=True)
        return np.array([np.random.choice(len(p), p=p) for p in policy])
    
    def logprobs(self, logits, actions):
        log_probs = F.log_softmax(logits)
        action_log_probs = log_probs.gather(1, actions)
        return action_log_probs
    
    def act(self, inputs):
        agent_outputs = model.forward(inputs)
        logits, state_values = agent_outputs
        actions = self.sample_actions(logits)
        action_log_probs = self.logprobs(logits, actions)
        return {
            'actions': actions,
            'logits': logits,
            'log_probs': action_log_probs,
            'values': state_values
        }
    # Should return a dict containing keys ['actions', 'logits', 'log_probs', 'values'].

Next will pass the environment and policy to a runner that collects partial trajectories from the environment. 
The class that does is is already implemented for you.

In [0]:
from util.runners import EnvRunner

This runner interacts with the environment for a given number of steps and returns a dictionary containing
keys 

* 'observations' 
* 'rewards' 
* 'resets'
* 'actions'
* all other keys that you defined in `Policy`

under each of these keys there is a python `list` of interactions with the environment of specified length $T$ &mdash; the size of partial trajectory. 

To train the part of the model that predicts state values you will need to compute the value targets. 
Any callable could be passed to `EnvRunner` to be applied to each partial trajectory after it is collected. 
Thus, we can implement and use `ComputeValueTargets` callable. 
The formula for the value targets is simple:

$$
\hat v(s_t) = \sum_{t'=0}^{T - 1}\gamma^{t'}r_{t+t'} + \gamma^T \hat{v}(s_{t+T}),
$$

In implementation, however, do not forget to use 
`trajectory['resets']` flags to check if you need to add the value targets at the next step when 
computing value targets for the current step. You can access `trajectory['state']['latest_observation']`
to get last observations in partial trajectory &mdash; $s_{t+T}$.

In [0]:
class ComputeValueTargets:
  def __init__(self, gamma=0.99):
    self.gamma = gamma
      
  def __call__(self, trajectory):
    # This method should modify trajectory inplace by adding 
    # an item with key 'value_targets' to it.
    value_targets = [0 if trajectory['resets'][-1] else trajectory['values'][-1]]
    for reward, done in zip(trajectory['rewards'], 
                            trajectory['resets'])[-2::-1]:
        if not done:
            value_targets.append(reward + self.gamma * value_targets[-1])
        else:
            value_targets.append(0)
    trajectory['value_targets'] = value_targets[::-1]
    return trajectory
    

After computing value targets we will transform lists of interactions into tensors
with the first dimension `batch_size` which is equal to `T * nenvs`, i.e. you essentially need
to flatten the first two dimensions. 

In [0]:
class MergeTimeBatch:
  """ Merges first two axes typically representing time and env batch. """
  def __call__(self, trajectory):
    # Modify trajectory inplace.
    for key, stack in trajectory.items():
        trajectory[key] = stack.reshape(-1, *stack.shape[2:])
    return trajectory
        

In [59]:
model = Agent("Agent", obs.shape, n_actions).to(device)
policy = Policy(model)
runner = EnvRunner(env, policy, nsteps=5,
                   transforms=[ComputeValueTargets(),
                               MergeTimeBatch()])

(8, 84, 84, 4)
linear input size =  2592


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: nn.init.orthogonal is now deprecated in favor of nn.init.orthogonal_.
  after removing the cwd from sys.path.


Now is the time to implement the advantage actor critic algorithm itself. You can look into your lecture,
[Mnih et al. 2016](https://arxiv.org/abs/1602.01783) paper, and [lecture](https://www.youtube.com/watch?v=Tol_jw5hWnI&list=PLkFD6_40KJIxJMR-j5A1mkxK26gh_qg37&index=20) by Sergey Levine.

In [0]:
class A2C:
  def __init__(self,
               policy,
               optimizer,
               value_loss_coef=0.25,
               entropy_coef=0.01,
               max_grad_norm=0.5):
    self.policy = policy
    self.optimizer = optimizer
    self.value_loss_coef = value_loss_coef
    self.entropy_coef = entropy_coef
    self.max_grad_norm = max_grad_norm
    
  def policy_loss(self, trajectory):
    # You will need to compute advantages here. 
    <TODO: implement>
    
  def value_loss(self, trajectory):
    <TODO: implement>
    
  def loss(self, trajectory):
    <TODO: implement>
      
  def step(self, trajectory):
    <TODO: implement>

Now you can train your model. With reasonable hyperparameters training on a single GTX1080 for 10 million steps across all batched environments (which translates to about 5 hours of wall clock time)
it should be possible to achieve *average raw reward over last 100 episodes* (the average is taken over 100 last 
episodes in each environment in the batch) of about 600. You should plot this quantity with respect to 
`runner.step_var` &mdash; the number of interactions with all environments. It is highly 
encouraged to also provide plots of the following quantities (these are useful for debugging as well):

* [Coefficient of Determination](https://en.wikipedia.org/wiki/Coefficient_of_determination) between 
value targets and value predictions
* Entropy of the policy $\pi$
* Value loss
* Policy loss
* Value targets
* Value predictions
* Gradient norm
* Advantages
* A2C loss

For optimization we suggest you use RMSProp with learning rate starting from 7e-4 and linearly decayed to 0, smoothing constant (alpha in PyTorch and decay in TensorFlow) equal to 0.99 and epsilon equal to 1e-5.

In [0]:
a2c = <Create instance of the algorithm> 

<Write your training loop>